## 混合混触危険性の予測（2024/2/23）

初期スクリプトに対して、可視化を容易にするため予測結果を危険性スコア（危険度を100点満点で表示）に変更。

In [20]:
import pandas as pd
import openai
from tqdm import tqdm
from src.utils.GPT import json_generate
from src.qra_gen import generate_reason_and_predict_mixture

In [21]:
#api_key
openai.api_key = "api_key"

In [22]:
#laod dataset as dict
csv_path="dataset/20240223_CRW_reactivity_sorted.csv"
df=pd.read_csv(csv_path)
df = df.iloc[:,:3]
chemical_records=df.to_dict(orient = "records")
chemical_records

[{'Substance 1': 'ALUMINUM OXIDE',
  'Substance 2': 'ACETYLENE',
  'Result': 'Caution'},
 {'Substance 1': 'ALUMINUM POWDER, UNCOATED',
  'Substance 2': 'ACETYLENE',
  'Result': 'Incompatible'},
 {'Substance 1': 'ALUMINUM POWDER, UNCOATED',
  'Substance 2': 'ALUMINUM OXIDE',
  'Result': 'Incompatible'},
 {'Substance 1': 'AMMONIA, ANHYDROUS',
  'Substance 2': 'ACETYLENE',
  'Result': 'Caution'},
 {'Substance 1': 'AMMONIA, ANHYDROUS',
  'Substance 2': 'ALUMINUM OXIDE',
  'Result': 'Compatible'},
 {'Substance 1': 'AMMONIA, ANHYDROUS',
  'Substance 2': 'ALUMINUM POWDER, UNCOATED',
  'Result': 'Incompatible'},
 {'Substance 1': 'ARGON', 'Substance 2': 'ACETYLENE', 'Result': 'Compatible'},
 {'Substance 1': 'ARGON',
  'Substance 2': 'ALUMINUM OXIDE',
  'Result': 'Compatible'},
 {'Substance 1': 'ARGON',
  'Substance 2': 'ALUMINUM POWDER, UNCOATED',
  'Result': 'Caution'},
 {'Substance 1': 'ARGON',
  'Substance 2': 'AMMONIA, ANHYDROUS',
  'Result': 'Compatible'},
 {'Substance 1': 'BENZENE',
  'Su

In [16]:
#このプロンプトを変更

# 質問文を生成するための指示
#qa_gen_command = """
#Provide the quantitative reasons within 300 words so that a scientist, who does not know the compatibility or reactivity of a mixture, can predict these outcomes.
#We must quantitatively consider how the combination of substances influences their compatibility or reactivity.
#I absolutely forbid you to make qualitative generalizations.

#Bad example reasons
## Its molecular weight compared to simpler compounds, contributing to higher reactivity. (qualitative discussion is practically meaningless!!!)
## Therefore, the mixture is highly reactive. (Never include the answer in the reason!!).

#Good example reasons
## Mixing sodium with water generates a significant amount of heat and hydrogen gas due to the chemical reaction, which can lead to an explosive reaction under certain conditions.
## The combination of acetone and bleach creates chloroform and other hazardous compounds due to the chemical reaction between acetone's methyl groups and sodium hypochlorite in bleach.

#Output: Reason key
#"""

qa_gen_command = """
Provide a detailed and quantitative analysis within 300 words to predict the compatibility or reactivity of a mixture on a scale from 0 to 100, where 100 indicates the highest level of hazard and 0 indicates no hazard. Focus specifically on the interactions between substances, including any potential reactions, and quantify their effects on the mixture's safety. Use scientific data and principles to support your prediction, avoiding broad generalizations.

#Bad example reasons
- "The molecular weight of this compound compared to simpler compounds suggests higher reactivity." (This is too general and does not provide specific interaction data or a clear link to the hazard score.)
- "Therefore, the mixture is highly reactive." (This statement is unsupported by quantitative data or specific conditions under which the reaction becomes hazardous.)

#Good example reasons
- "Mixing sodium with water is expected to result in a hazard score of 95 due to an exothermic reaction that increases the temperature of the solution up to 100°C within 30 seconds, releasing 56.6 kJ/mol of energy. This rapid energy release can cause boiling and splattering, posing a burn hazard."
- "The combination of acetone and bleach at a ratio of 1:1 by volume is predicted to have a hazard score of 80. The reaction yields chloroform at a 20% reaction yield based on acetone volume, with chloroform concentrations exceeding 0.002 ppm in air posing neurotoxic and carcinogenic risks."
- "Mixing water with sugar is predicted to have a hazard score of 5, as this combination results in a simple solution without any hazardous reactions. The dissolution of sugar in water is an endothermic process that slightly decreases the temperature of the solution, posing no safety risk under normal handling and storage conditions."

#Output: Reason key
In your analysis, include considerations such as the specific chemical reactions, concentration thresholds, reaction rates, and environmental factors that either mitigate or exacerbate risks. Your prediction should reflect how these factors contribute to the overall hazard score of the mixture.
Consider not only the potential for a reaction but also the conditions under which the reaction is safe or hazardous. Include considerations such as concentration thresholds, reaction rates, and environmental factors that mitigate or exacerbate risks.
"""

# 分子構造から物性を予測するための指示
#predict_command = """
#Predict the compatibility or reactivity of the following mixture.
#Consider the quantitative reasons provided and, in any case, output a prediction about the mixture's safety or potential hazards.

#Output: Result key
#"""

#predict_command = """
#Predict the compatibility or reactivity of the following mixture, using only one of the following outcomes: Compatible, Incompatible, Caution. 
#Consider the quantitative reasons provided and make your prediction based on the mixture's safety or potential hazards.

#Output: Result key should be one of [Compatible, Incompatible, Caution].
#"""

predict_command = """
Predict the level of danger of the following mixture on a scale of 0 to 100, where 0 indicates no danger at all and 100 represents the highest level of danger. 
Consider the quantitative reasons provided and make your prediction based on the mixture's safety or potential hazards. Provide your prediction as a numerical value reflecting the mixture's level of danger.

#Output: Result key should be a numerical value between 0 and 100.
"""

#use GPT3.5 completion
llm_ask_func=json_generate

#if gen_reason=True, generate reason and predict
#if gen_reason=False, only predict without reason (for control)
gen_reason=True
#gen_reason=False

#n回のpredictionを行う
n_trials=3

In [17]:


completed_records = []
for record in tqdm(chemical_records):  # tqdmは進捗状況を表示
    try:
        print("Hello")
            # generate_reason_and_predict関数により、理由生成と予測実行
        completed_record = generate_reason_and_predict_mixture(
            substance1=record["Substance 1"],
            substance2=record["Substance 2"],
            result=record["Result"],  # resultは混合物の危険性判定の結果
            qa_gen_command=qa_gen_command,  # 理由を尋ねるための指示
            predict_command=predict_command,  # 予測を行うための指示
            llm_ask_func=json_generate,  # LLM関数
    #            smiles=record.get("smiles", ""),  # SMILES情報（存在する場合）
            gen_reason=gen_reason,  # 理由を生成するかどうか
            n_trials=n_trials  # 予測を試みる回数
        )
#        print(completed_record)
    except Exception as e:
        print("negative")
#        traceback.print_exc()
        print(e)  # 例外が発生した場合にエラーを表示
        completed_record = record  # 例外が発生した場合は元のレコードを使用

    completed_records.append(completed_record)  # 処理されたレコードをリストに追加


  0%|                                                                                         | 0/1328 [00:00<?, ?it/s]

Hello
{'Result': 15}
{'Result': 15}


  0%|                                                                               | 1/1328 [00:05<1:50:47,  5.01s/it]

{'Result': 20}
Hello
{'Result': 85}
{'Result': 85}


  0%|                                                                               | 2/1328 [00:08<1:28:48,  4.02s/it]

...

{'Result': 10}
Hello
{'Result': 85}
{'Result': 90}


100%|███████████████████████████████████████████████████████████████████████████▉| 1327/1328 [2:15:02<00:03,  3.34s/it]

{'Result': 85}
Hello
{'Result': 90}
{'Result': 90}


100%|████████████████████████████████████████████████████████████████████████████| 1328/1328 [2:15:06<00:00,  6.10s/it]

{'Result': 90}


In [18]:
completed_records

[{'substance1': '2-CHLORO-P-PHENYLENEDIAMINE SULFATE',
  'substance2': '1-BUTENE',
  'result': 'Compatible',
  'generated_reason': '2-CHLORO-P-PHENYLENEDIAMINE SULFATE and 1-BUTENE are predicted to be compatible based on their chemical properties and potential interactions. While 2-CHLORO-P-PHENYLENEDIAMINE SULFATE is an aromatic amine compound commonly used in hair dyes, 1-BUTENE is a simple olefin commonly used in the production of plastics and synthetic rubber. These substances are unlikely to undergo a rapid or hazardous reaction when mixed, as they do not possess functional groups that are highly reactive with each other. Additionally, there are no known exothermic reactions or toxic by-products resulting from the combination of these substances. Therefore, based on their chemical structures and lack of significant reactivity, the mixture of 2-CHLORO-P-PHENYLENEDIAMINE SULFATE and 1-BUTENE is not expected to pose a safety risk under normal handling and storage conditions.',
  'pre

In [19]:
#save
import datetime
import json

study_name="mixture_practice_w_reason_numeric_train"
now=datetime.datetime.now()
now_str=now.strftime("%Y%m%d_%H%M%S")
save_path=f"results/{study_name}_{now_str}.json"
with open(save_path, 'w', encoding = "utf-8") as f:
    json.dump(completed_records, f, indent=4, ensure_ascii=False)